# Project 2 | Classification of Skin Lesions
### Classifying skin lesions using scipy, sklearn and cv2

This notebook contains a step by step process to recreate the results of the report.

A detailed explanation regarding feature extraction and choices of classification model can be found in the report.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import path
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import cv2
from sklearn.preprocessing import StandardScaler
import groupXY_functions as util
import os

# Feature extraction

- load segmenation mask and lesion image
- extract features
- write to csv file

In [ ]:
## DISCLAMER: All the code for feature extraction is in groupXY_functions.py, and imported as util.
## if you need more explenation of the code, this is a good place to look for quality comments 
# The classify script is in two scripts: classify.py for classic approach and classify_cnn.py for neural networks

In [ ]:
feature_extract = False

In [ ]:
#reading groundtruth for the full 2000 pictures 
if feature_extract: dfout = pd.read_csv(path.ground_truth_full)

In [ ]:
if feature_extract:
	for index, line in dfout.iterrows():
		# for each image in ground truth, load img, mask
		id_ = line['image_id'] 
		
		img = cv2.imread(path.images_full+f'{id_}.jpg')
		mask = cv2.imread(path.segmentation_full+f'{id_}_segmentation.png',0)

		img, mask = util.scale_image_res(img,mask)
		
		img_plt, mask_plt = plt.imread(path.images_full+f'{id_}.jpg'), plt.imread(path.segmentation_full+f'{id_}_segmentation.png')
		img_plt, mask_plt = util.scale_image_res(img_plt,mask_plt)	
	
		# remove skin from lesion pictures.
		res = cv2.bitwise_and(img,img,mask = mask)
		notmask = cv2.bitwise_not(mask)
		notres = cv2.bitwise_and(img,img,mask = notmask)


		print(f'Running extraction on image {index} out of {dfout.shape[0]} ({index / dfout.shape[0] : 0.2%})', end='\r', flush=True)

		# add feature to dfout
		dfout.at[index, ['color_lesion_r','color_lesion_g','color_lesion_b']] = util.averageColorsSimple(res)

		dfout.at[index, ['color_skin_r','color_skin_g','color_skin_b']] = util.averageColorsSimple(notres)

		dfout.at[index, ['area','perimeter']] = util.measure_area_perimeter(mask)

		dfout.at[index, ['min_symmetry', 'avg_symmetry']] = util.all_deg(mask)

		# dfout.at[index, ['border_dark', 'border_skin']] = util.boundry_check2(img_plt,mask_plt)

		dfout.at[index, ['color_variance']] = util.color_variance(img_plt, mask_plt)

In [ ]:
if feature_extract:
    save_path = path.out+'out_rescaled.csv'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    dfout.to_csv(save_path)

In [ ]:
df = pd.read_csv(path.out+'out_rescaled.csv', index_col=0)
df.drop(['seborrheic_keratosis'], axis=1, inplace=True)
df.set_index('image_id', inplace = True)
df

## Note
Min and avg symmetry is based on amount of pixels that dont overlap at different rotations. To account for different sizes of masks we devide by the area to standardize the scores and give a procent of the masks that dont overlap when folded.

In [ ]:
# Standardize symmetry 
df['std_min_symmetry'] = df['min_symmetry'] / (df['area'] / 255)
df['std_avg_symmetry'] = df['avg_symmetry'] / (df['area'] / 255)

# Due to how we deal with masks hiiting border their area was taken based on sum of pixels with value 255.
# So here we correct by setting all values of 255 to 1 as it would be since area / area = 1
df.loc[ df['std_min_symmetry'] > 254, 'std_min_symmetry'] = 1
df.loc[ df['std_avg_symmetry'] > 254, 'std_avg_symmetry'] = 1


To capture some of the irregularities in the border we added the formula for circularity

In [ ]:
df['circularity'] = (4*np.pi*df['area'])/(df['perimeter'])**2

In [ ]:
df.drop(['min_symmetry', 'avg_symmetry', 'area', 'perimeter', 'color_lesion_b', 'color_lesion_g', 'color_skin_g', 'color_skin_b'], axis=1, inplace=True)

Scaled with standard scalar

In [ ]:
#scaling features with standardscalar
featureDf = df.drop(['melanoma'], axis=1)
#logfeature = np.log(featureDf)

scalar = StandardScaler().fit(featureDf)

In [ ]:
scaled_features = scalar.transform(featureDf)

In [ ]:
#combining scaled df with our Y values
scaled_features_df = pd.DataFrame(scaled_features, index=featureDf.index, columns=featureDf.columns)
finalDf = pd.concat([scaled_features_df, df['melanoma']], axis = 1)

In [ ]:
#colors = {0:'tab:blue', 1:'tab:red'}
#sns.pairplot(finalDf, hue='melanoma')

In [ ]:
from itertools import combinations
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
df2=finalDf

X = df2.drop(['melanoma'], axis=1,)

Y = df2['melanoma']

## CODE TO CHECK ALL FEATURE COMBINATIONS BASED ON MEAN AUC ON 500 ITERATIONS ##

# features = list(X.columns)

# k = 3
# best_auc = 0

# for i in range(2, 7):
#     print(i)
#     for comb in tqdm(combinations(features, i)):
#         inner_auc = []
#         inner_acc = []
#         for j in range(500):
#             X_dev, X_test, y_dev, y_test = train_test_split(
#                     X[list(comb)], y, stratify=y)

#             X_train, X_val, y_train, y_val = train_test_split(
#                     X_dev, y_dev, stratify=y_dev)

#             knn = KNeighborsClassifier(n_neighbors=k) # other hyperparameters possible
#             knntrained = knn.fit(X_train, y_train)

#             y_val_knn = knntrained.predict(X_val)

#             acc_knn = accuracy_score(y_val, y_val_knn)
#             auc_knn = roc_auc_score(y_val, y_val_knn)

#             inner_auc.append(auc_knn)
#             inner_acc.append(acc_knn)
        
#         inner_auc = [1 - thing if thing < 0.5 else thing for thing in inner_auc]
#         mean_auc = np.mean(inner_auc)
#         mean_acc = np.mean(inner_acc)

#         if mean_auc > best_auc:
#             best_auc = mean_auc
#             best_acc = mean_acc
#             best_comb = comb

In [ ]:
# Split dataset to development and test set
X_dev, X_test, y_dev, y_test = train_test_split(
        X[['color_variance', 'std_avg_symmetry']], Y, stratify=Y, test_size=0.2, random_state=180)

# Split development set to training and valuation sets

X_train, X_val, y_train, y_val = train_test_split(
       X_dev, y_dev, stratify=y_dev, random_state=180)

In [ ]:
# auc_list = [1 - thing if thing < 0.5 else thing for thing in auc_list]
# np.mean(auc_list)

In [ ]:
# Train a classifiers
knn = KNeighborsClassifier(n_neighbors=5)
knntrained = knn.fit(X_train, y_train)

dtree = DecisionTreeClassifier()
dtree = dtree.fit(X_train, y_train)

y_val_tree = dtree.predict(X_val)
y_val_knn = knntrained.predict(X_val)

from sklearn.model_selection import cross_val_score
scores_knn = cross_val_score(knntrained, X_val, y_val, cv=6)
scores_tree = cross_val_score(dtree, X_val, y_val, cv=6)

auc_knn = roc_auc_score(y_val, y_val_knn)
auc_tree = roc_auc_score(y_val, y_val_tree)


f1_knn = f1_score(y_val, y_val_knn)
f1_tree = f1_score(y_val, y_val_tree)

print(f'KNN: \nAccuracy: {np.mean(scores_knn)}  AUC_ROC: {auc_knn}  F1: {f1_knn}\n')
print(f'Tree: \nAccuracy: {np.mean(scores_tree)}  AUC_ROC: {auc_tree} F1: {f1_tree}')



In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#Replication of KNN model
cm = confusion_matrix(y_val, y_val_knn, labels=[0, 1])
print(cm)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

#The Chart of CNN model is available in cnn_evaluation file

In [ ]:
# ONLY FOR REPORTING, also evaluate on test set
X_test = X_test

y_test_knn = knntrained.predict(X_test)
y_test_tree = dtree.predict(X_test)

auc_test = roc_auc_score(y_test, y_test_knn)
auc_test_tree = roc_auc_score(y_test, y_test_tree)

print(auc_test)
print(auc_test_tree)

# For sma
#ll datasets these results will depend on the random seed you chose when splitting, 
# this is why it is good to look at multiple splits/cross-validation

In [ ]:
import pickle
with open('pickle_knn.pickle','wb') as pickle_out:
    pickle.dump(knntrained,pickle_out)

with open('pickle_scalar.pickle','wb') as scalar_out:
    pickle.dump(scalar,scalar_out)

In [ ]:
#This function is in its entirety written and described in GroupXY_functions as Color_variance
#This part only reproduced the graphics used in the report

path = "ISIC_0007235"

image = plt.imread(f'data/example_image/{path}.jpg')
mask = plt.imread(f'data/example_segmentation/{path}_segmentation.png')
plt.imshow(mask, cmap='gray')
image, mask = util.scale_image_res(image, mask)
(h, w) = image.shape[:2]

_, mask2 = util.extract_mask_roi(image, mask, 20, 20)
image[mask2==0] = 0 #if the mask is black at some spot, set image color to black at that spot

plt.imshow(image)

In [ ]:
from sklearn.cluster import MiniBatchKMeans

image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

image = image.reshape((image.shape[0] * image.shape[1], 3))

# apply k-means using the specified number of clusters (16) and
# then create the quantized image based on the predictions

clt = MiniBatchKMeans(n_clusters = 16, random_state=3)
labels = clt.fit_predict(image)
quant = clt.cluster_centers_.astype("uint8")[labels]

# reshape the feature vectors to images
quant = quant.reshape((h, w, 3))

# convert from L*a*b* to BGR and then to RGB gor matplotlib plotting
quant = cv2.cvtColor(quant, cv2.COLOR_LAB2BGR)
quant = cv2.cvtColor(quant, cv2.COLOR_BGR2RGB)

#graph = np.hstack([image, quant]) #hstack just makes them appear one after another for comparison
plt.imshow(quant)
plt.tight_layout()

In [ ]:
#Replication of Symmetry graphics at one given degree. The entire function is available in GroupXY_functions

mask = cv2.imread(f'data/example_segmentation/ISIC_0001769_segmentation.png',0)
a,b,c,d = util.extract_bboxes(np.array(mask))
img = mask[a:b,c:d]
center = util.middle(img)
first_half = img[:,:int(center[0])]
second_half = img[:,int(center[0]):-1]
_, first_size = first_half.shape
_, second_size = second_half.shape
if first_size > second_size:
    padd_rows = first_size-second_size
    second_half = np.pad(second_half,((0,0),(0,padd_rows)),"constant",constant_values=(0))
elif first_size < second_size:
    padd_rows = abs(first_size-second_size)
    first_half = np.pad(first_half,((0,0),(padd_rows,0)),"constant",constant_values=0)
flipped = cv2.flip(first_half, 1)
sum_mask = np.array(flipped) ^ np.array(second_half)

fig, ax = plt.subplots(ncols = 4)

ax[0].imshow(img)
ax[1].imshow(flipped)
ax[2].imshow(second_half)
ax[3].imshow(sum_mask)